### RQ1

**Build the graph G=(V, E), where V is the set of articles and E the hyperlinks among them, and provide its basic information:** 

**1. If it is direct or not?**

To answer this question we have to add edges and nodes to the networkx librarary. There is function called **nx.is_directed(G)** whether the graph is directed or not. 

In [1]:
#adding all the libraries
import pandas as pd
import numpy as np
import networkx as nx
import collections
from os.path import isfile
from statistics import median

In [2]:
# adding functions file. 
from hw05_FUNCTIONS import *

Now we are going to use simple graph first to check whether it is directed or not. 

In [8]:
G = nx.DiGraph()
f = open("data/wiki-topcats-page-names.txt", "r")
for row in f:
    lrow = row.split()
    name = ' '.join(lrow[1:])
    G.add_node(int(lrow[0]))
    G.node[int(lrow[0])]["name"] = name
f.close()

In [4]:
G.node[1]

{'name': 'Kleroterion'}

After adding nodes we have to add edges for these lets use **wiki-topcats-reduced.txt** file so that we have insights about the graph. 

In [9]:
flink = open("data/wiki-topcats-reduced.txt", "r")
for i,row in enumerate(flink):
    lrow = row.split()
    G.add_edge(int(lrow[0]), int(lrow[1]))
flink.close()

In [7]:
nx.is_directed(G) # this function will tell us whether it is directed or not. 

True

*This tells us that graph we are trying to work with is directed because edge meaning <node1,node2> in node1 wikipedia article got a link to the node2 wikipedia article.* so this answer the question whether the graph is directed or not.* **The graph is directed**

### The number of nodes? The number of edges? 
for this we are going to use simple functions like **number_of_nodes** and **number_of_edges()**. 

In [11]:
G.number_of_nodes()

1791489

In [12]:
G.number_of_edges()

2174451

The number of nodes are **1791489** and number of edges are **2174451**. 

### The average node degree. Is the graph dense?

To answer this we will use *density()** and degree function. 

In [13]:
degreeM = 0
for node in G.node:
    degreeM += G.degree(node)
print(degreeM/G.number_of_nodes())

2.4275348606661833


In [14]:
nx.density(G)

1.3550383037263901e-06

So to answer above question the average node degree is *2.4275348606661833* and density is *1.3550383037263901e-06*. which says that the graph is not much dense. 

Lets move to reasearch question 2. 

## RQ2:
Given a category C_0 = \{article_1, article_2, \dots \} as input we want to rank all of the nodes in V according to the given criteria discussed in given question. 


First we have to define constants so that we can work with our defined functions. 

In [10]:
#this is the folder path where all the data are stored
dataFolder = "./data/"
#base filename for all wiki data
basefilename = "wiki-topcats-"
#file extension of wiki data
ext = ".txt"
#name of all wiki data files
CATEGORIES = "categories"
REDUCED_GRAPH = "reduced"
PAGE_NAMES = "page-names"

let's open the wiki_categories.txt file and delete all categories with less than 3500 articles

In [12]:
#read the category file
categories = pd.read_csv(dataFolder+basefilename+CATEGORIES+ext, sep = ";", header = None , names = ["Category","List_of_articles"])
#clean the category column
#Before: "Category: category_name"
#After: "category_name"
categories["Category"] = categories["Category"].apply(lambda x:x.split(":")[1])
#defining a function that delete all categories with less than 3500 articles
ARTICLES_THRESHOLD = 3500
#Scanning each value of list_of_articles, if the number of articles goes beyond the threshold then retain it
#otherwise return an empty string
categories["List_of_articles"] = categories["List_of_articles"].apply(lambda x: x if len(x.strip().split(" ")) >= ARTICLES_THRESHOLD else "")
#delete all rows with empty string as "list_of_articles"
categories = categories[categories["List_of_articles"] != ""]

Now it is needed to build the final graph, but it must be checked that the nodes into the reduced graph and 
1. the nodes into the categories files are the same.
2. The set of the nodes into the categories must be built.
3. The set of nodes into the reduced graph must be built.
4. the intersection between these two sets must be computed.
5. only the edges involving nodes of the intersection set must be added to the final graph

In [13]:
#let's open the reduced graph file and create the set of nodes 
reduced_graph = pd.read_csv(dataFolder+basefilename+REDUCED_GRAPH+ext, sep = "\t", header = None, names = ["Node_1","Node_2"])
#create the set of the first column
set_node_1 = set(reduced_graph["Node_1"].values.tolist())
#create the set of the second column
set_node_2 = set(reduced_graph["Node_2"].values.tolist())
#create the set of the nodes into the reduced graph through the union operation of the two previous created sets
set_reduced_graph_nodes = set.union(set_node_1,set_node_2)
#print the size for debugging
print("size of set_node_1:= "+str(len(set_node_1)))
print("size of set_node_2:= "+str(len(set_node_2)))
print("size of set_reduced_graph_nodes:= "+str(len(set_reduced_graph_nodes)))
#The two previous sets are not useful anymore therefore they are deleted from the main memory
del set_node_1
del set_node_2

#Let's create the set of nodes into the categories
#create initially an empty set
set_categories_nodes = set()
#in order to perform the intersection function the nodes must be represented in the same format
#since the nodes into the @reduced_graph dataframe are integer then the "int" type is chosen
categories["List_of_articles"].apply(lambda x: set_categories_nodes.update(set(map(int, x.strip().split(" ")))))

#compute the final set of nodes
set_of_nodes = set.intersection(set_categories_nodes, set_reduced_graph_nodes)
#print the size for debugging
print("size of set_categories_nodes:= "+str(len(set_categories_nodes)))
print("size of set_reduced_graph_nodes:= "+str(len(set_reduced_graph_nodes)))
print("size of set_of_nodes:= "+str(len(set_of_nodes)))
#once the final set is computed the other sets can be deleted to free the memory
del set_categories_nodes
del set_reduced_graph_nodes

size of set_node_1:= 428957
size of set_node_2:= 352518
size of set_reduced_graph_nodes:= 461193
size of set_categories_nodes:= 546237
size of set_reduced_graph_nodes:= 461193
size of set_of_nodes:= 461193


Now it is possible to read line by line all the edges of the reduced graph and adding it to the final graph only if they belong to the @set_of_nodes computed

In [ ]:
final_graph = nx.DiGraph()

print(".")
#build the graph
reduced_graph.apply(lambda edge: final_graph.add_edge(edge[0],edge[1]) if filterEdges(edge,set_of_nodes) else "" ,axis = 1)

Now that we have build the graph we must filter out the nodes in the category so that we only have good nodes. For that we have used **filter_nodes_in_categories** in **hw05_FUNCTIONS.py** file. 

In [15]:
#the category dataframe must involve only "good nodes" too
categories["List_of_articles"] = categories["List_of_articles"].apply(lambda x: filter_nodes_in_categories(x,set_of_nodes))

After that we are going to use function named **retrieveCategoryDistances** for getting the categories distance. This function also uses **computeCategoryDistance** for computing the distance for each category which also internally uses **breadth_first_search** function which is being used to actually calculating the distance for each node to all other nodes. After that we will have distances for each category. 

In [17]:
category_distances = retrieveCategoryDistances(filename = "category_distances.npy", final_graph = final_graph ,categories=categories)

We will compute a dictionary that has as keys the categories and as values the list of articles that belongs to that category

In [19]:
category_dictionary = {}
#for each category
for i in range(categories.shape[0]):
    #assign to that category the list of articles
    category_dictionary[i] = list(map(int, categories.iloc[i]['List_of_articles'].split(" ")))

After that we will just compute the block ranking. Its just sorting of the category_distances dictionary values. We will use **computeBlockRanking** function for that. 

In [22]:
blockRanking = computeBlockRanking(0,category_distances)

Now we will compute nodes ranking using final graph, block rankings and categoris of dictionary and outputing it to the file at the same time displaying top ten nodes here. 

In [23]:
compute_nodes_ranking(allGraph = final_graph, block_ranking = blockRanking, categories = category_dictionary)

The top-10 ranked articles are the following: 
81941 82322 82082 82089 82091 81871 81878 82346 82084 81267
The entire ranking is saved on the filesystem
